# Ch 6: Data Loading, Storage, and File Formats

In [1]:
# conda install lxml
# pip install beautifulsoup4 html5lib

In [2]:
import pandas as pd
import sys
import numpy as np
import csv
import json
from lxml import objectify
from io import StringIO
import requests
import sqlalchemy as sqla

## 6.1 Reading and Writing Data in Text Format

In [3]:
!cat ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [4]:
!type ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


Comma-delimited; can use **read_csv**:

In [5]:
df = pd.read_csv('ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


No header:

In [6]:
!cat ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [7]:
pd.read_csv('ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
pd.read_csv('ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Make the 'message' column the index:

In [9]:
names=['a', 'b', 'c', 'd', 'message']

In [10]:
pd.read_csv('ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


Pass a list of column numbers/names to form a heirarchical index from multiple columns:

In [11]:
!cat csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [12]:
parsed = pd.read_csv('csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

Sometimes, a table might not have a fixed delimiter:

In [13]:
list(open('ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

Notice the fields are separated by variable amounts of whitespace.  
Can pass a regular expression as a delimiter: \\s+

In [14]:
result = pd.read_csv('ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [15]:
!cat ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


Skip the 1st, 3rd, and 4th rows using `skiprows`:

In [16]:
pd.read_csv('ex4.csv', skiprows=[0, 2, 3], index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


Handle missing values:

In [17]:
!cat ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [18]:
result = pd.read_csv('ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [19]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [20]:
result = pd.read_csv('ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Can specify different NA sentinels for each column in a dict:

In [21]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}

In [22]:
pd.read_csv('ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in Pieces

First, let's make the pandas display settings more compact:

In [23]:
pd.options.display.max_rows = 10

In [24]:
result = pd.read_csv('ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


Use `nrows` to read a small number of rows:

In [25]:
pd.read_csv('ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


Read in pieces using `chunksize`:

In [26]:
chunker = pd.read_csv('ex6.csv', chunksize=1000)
chunker

Iterate over the file, aggregating the value counts in the 'key' column:

In [27]:
chunker = pd.read_csv('ex6.csv', chunksize=1000)

tot = pd.Series([], dtype=float)
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

In [28]:
tot

0    151.0
1    146.0
2    152.0
3    162.0
4    171.0
     ...  
V    328.0
W    305.0
X    364.0
Y    314.0
Z    288.0
Length: 36, dtype: float64

In [29]:
tot = tot.sort_values(ascending=False)

In [30]:
tot

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

### Writing Data to Text Format

In [31]:
data = pd.read_csv('ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Use `to_csv` to write to a csv file:

In [32]:
data.to_csv('out.csv')

In [33]:
!cat out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


Other delimiters can be used:

In [34]:
import sys

In [35]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Can specify sentinel value using `na_rep`:

In [36]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


Can also disable row and column labels by setting `index` and `header` to **False**:

In [37]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


Or, just write a subset of the columns:

In [38]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Can also use `to_csv` on a **Series**:

In [39]:
import numpy as np

In [40]:
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('tseries.csv')

In [41]:
!cat tseries.csv

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### Working with Delimited Formats

In [42]:
!cat ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


Can use Python's built-in **csv** module with single-character delimiters:

In [43]:
import csv

In [44]:
f = open('ex7.csv')
reader = csv.reader(f)

In [45]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


First, read the file into a list of lines:

In [46]:
with open('ex7.csv') as f:
    lines = list(csv.reader(f))

Split the lines into header and data lines:

In [47]:
header, values = lines[0], lines[1:]

Create a dictionary of data columns using dictionary comprehension and `zip(*values)` expression, which transposes rows to columns:

In [48]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

In [49]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

Define a subclass of `csv.Dialect`:

In [50]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

# reader = csv.reader(f, dialect=my_dialect)

In [51]:
type(f)

_io.TextIOWrapper

In [52]:
# reader = csv.reader(f, delimiter-'|')

Can also write delimited files manually using `csv.writer` (similar to `csv.reader`):

In [53]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

### JSON Data

In [54]:
obj = """
{
 "name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [55]:
type(obj)

str

Use `json.loads` to read a JSON string into Python:

In [56]:
import json

In [57]:
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [58]:
type(result)

dict

Convert from Python back to JSON using `json.dumps`:

In [59]:
asjson = json.dumps(result)

In [60]:
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [61]:
print(json.dumps(result, indent = 3)) 

{
   "name": "Wes",
   "places_lived": [
      "United States",
      "Spain",
      "Germany"
   ],
   "pet": null,
   "siblings": [
      {
         "name": "Scott",
         "age": 30,
         "pets": [
            "Zeus",
            "Zuko"
         ]
      },
      {
         "name": "Katie",
         "age": 38,
         "pets": [
            "Sixes",
            "Stache",
            "Cisco"
         ]
      }
   ]
}


In [62]:
type(asjson)

str

Pass a list of dicts (from JSON) to the DF constructor and select a subset of data fields:

In [63]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [64]:
!cat example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


By default `read_json` assumes each object in the JSON array is a row:

In [65]:
data = pd.read_json('example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


Can also export data from pandas to JSON using `to_json`:

In [66]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [67]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### XML and HTML: Web Scraping

In [68]:
# conda install lxml

In [69]:
# pip install beautifulsoup4 html5lib

In [70]:
tables = pd.read_html('fdic_failed_bank_list.html')

In [77]:
type(tables)

list

In [72]:
len(tables)

1

In [74]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [78]:
type(failures)

pandas.core.frame.DataFrame

In [76]:
len(failures)

547

In [79]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [80]:
close_timestamps.head()

0   2016-09-23
1   2016-08-19
2   2016-05-06
3   2016-04-29
4   2016-03-11
Name: Closing Date, dtype: datetime64[ns]

In [81]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

In [82]:
type(close_timestamps.dt.year.value_counts())

pandas.core.series.Series

### Parsing XML with `lxml.objectify`

In [83]:
from lxml import objectify

In [84]:
path = 'mta_perf/Performance_MNR.xml'

In [85]:
parsed = objectify.parse(open(path))

In [87]:
type(parsed)

lxml.etree._ElementTree

In [88]:
root = parsed.getroot()

In [90]:
type(root)

lxml.objectify.ObjectifiedElement

In [91]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

Returns a list of dicts.

In [97]:
type(data)

list

In [96]:
type(data[0])

dict

In [94]:
len(data)

648

Convert the list of dicts into a DF:

In [98]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95,96.9,95,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95,96,95,95
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95,96.3,95,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95,96.8,95,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95,96.6,95,95.8


In [99]:
len(perf)

648

Each XML tag can have metadata. An HTML tag is also valid XML

In [100]:
from io import StringIO

In [101]:
tag = '<a href="https://www.google.com/">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

Can now access any of the fields (like href) in the tag or link text:

In [102]:
root

<Element a at 0x262054b9d00>

In [103]:
root.get('href')

'https://www.google.com/'

In [104]:
root.text

'Google'

## 6.2 Binary Data Formats

Store data in a binary format using Python's `pickle` serialization:

In [105]:
frame = pd.read_csv('ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [106]:
frame.to_pickle('frame_pickle')

Read a "pickled" object using `pickle` directly, or using `pandas.read_pickle`:

In [107]:
pd.read_pickle('frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### Using HDF5 Format

The `HDFStore` class works like a dict and handles low-level details:

In [112]:
frame = pd.DataFrame({'a': np.random.randn(100)})

In [115]:
type(frame)

pandas.core.frame.DataFrame

In [118]:
frame

,a
0,-0.177818
1,-1.343322
2,-0.468487
3,-0.500663
4,-1.440570
...,...
95,0.615493
96,0.683111
97,-1.082299
98,0.208180


In [113]:
store = pd.HDFStore('mydata.h5')

In [116]:
type(store)

pandas.io.pytables.HDFStore

In [119]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [117]:
store['obj1'] = frame

In [120]:
store['obj1_col'] = frame['a']

Objects contained in the HDF5 file can be retrieved w/dict-like API:

In [122]:
store['obj1']

,a
0,-0.177818
1,-1.343322
2,-0.468487
3,-0.500663
4,-1.440570
...,...
95,0.615493
96,0.683111
97,-1.082299
98,0.208180


In [123]:
store.put('obj2', frame, format='table')

In [125]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-0.032561
11,-0.016854
12,0.974564
13,1.701537
14,0.848636
15,0.162281


`put` is an explicit version of `store['obj2'] = frame` but allows other options, like storage format.  
`pandas.read_hdf` gives a shortcut to these tools:

In [126]:
frame.to_hdf('mydata.h5', 'obj3', format='table')

In [132]:
# pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

In [133]:
# os.remove('mydata.h5')

### Reading Microsoft Excel Files

To use `ExcelFile`, create an instance by passinga  path to a file:

In [134]:
xlsx = pd.ExcelFile('ex1.xlsx')

Data stored in a sheet can be read into a DF with `parse`:

In [135]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


If reading multiple Sheets, it's faster to create the ExcelFile first, but you can also read a sheet directly:

In [136]:
frame = pd.read_excel('ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


To write pandas to Excel, first create an `ExcelWriter`, then write data to it using `to_excel`:

In [137]:
writer = pd.ExcelWriter('ex2.xlsx')

In [138]:
frame.to_excel(writer, 'Sheet1')

In [139]:
writer.save()

You can also pass a file path to `to_excel` and avoid using the `ExcelWriter':

In [141]:
frame.to_excel('ex2.xlsx')

## 6.3 Interacting with Web APIs

Find the last 30 GitHub issues for pandas by making a `GET` HTTP request using the `requests` library:

In [142]:
import requests

In [143]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

In [146]:
resp = requests.get(url)
resp

<Response [200]>

Use the `json` method from `response` to return a dict containing JSON parsed into Python objects:

In [149]:
data = resp.json()

In [150]:
len(data)

30

In [152]:
data[0]['title']

'DEPR: astype dt64<->dt64tz'

In [153]:
type(data)

list

In [154]:
type(data[0])

dict

In [155]:
type(data[0]['title'])

str

In [157]:
data[0].keys()

dict_keys(['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'performed_via_github_app'])

Pass **data** directly to DF and extract fields of interest:

In [158]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])

In [159]:
issues

,number,title,labels,state
0,39258,DEPR: astype dt64<->dt64tz,[],open
1,39257,CLN refactor maybe-castable,[],open
2,39256,DOC: keep color for note admonitions in sphinx...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
3,39253,REGR: codecs.open() is always opened in text mode,"[{'id': 2301354, 'node_id': 'MDU6TGFiZWwyMzAxM...",open
4,39250,BUG: read_excel fails with IndexError: list in...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
...,...,...,...,...
25,39212,BUG: DataFrame.apply axis=1 for str ops with n...,"[{'id': 697792067, 'node_id': 'MDU6TGFiZWw2OTc...",open
26,39211,BUG: DataFrame.apply axis=1 for str ops with n...,"[{'id': 697792067, 'node_id': 'MDU6TGFiZWw2OTc...",open
27,39209,API: Should apply be smart?,"[{'id': 35818298, 'node_id': 'MDU6TGFiZWwzNTgx...",open
28,39207,DOC: add new page 'how to read these doc' in g...,[],open


## 6.4 Interacting with Databases

Create a SQLite db using Python’s built-in `sqlite3` driver:

In [160]:
import sqlite3

In [161]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
 );
"""

In [163]:
con = sqlite3.connect('mydata.sqlite')

In [164]:
type(con)

sqlite3.Connection

In [165]:
con.execute(query)

In [166]:
con.commit() # basically 'Save'

Insert a few rows of data:

In [167]:
data = [('Atlanta', 'Georgia', 1.25, 6),
       ('Tallahassee', 'Florida', 2.6, 3),
       ('Sacramento', 'California', 1.7, 5)]

In [168]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [169]:
con.executemany(stmt, data)

In [170]:
con.commit()

Most Python SQL drivers return a list of tuples when selecting data from a table:

In [171]:
cursor = con.execute('select * from test')

In [172]:
type(cursor)

sqlite3.Cursor

In [173]:
rows = cursor.fetchall()

In [175]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

You can pass a list of tuples to DF constructor, but you need the column names from the cursor's `description` attribute:

In [176]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [178]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


Connect to the same SQLite database with SQLAlchemy and read data from the table created before:

In [179]:
import sqlalchemy as sqla

In [180]:
db = sqla.create_engine('sqlite:///mydata.sqlite')

In [181]:
type(db)

sqlalchemy.engine.base.Engine

In [182]:
db

Engine(sqlite:///mydata.sqlite)

In [183]:
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


## 6.5 Conclusion

In [ ]:
# csvdata["Hispanic"]=round((csvdata["Hispanic"]*csvdata["TotalPop"])/100, 0)

In [ ]:
# def pertonum(col):
#     csvdata[col]=round((csvdata[col]*csvdata["TotalPop"])/100, 0)

In [184]:
def bounded_repeater(value, max_repeats):
    for i in range(max_repeats):
        yield value

iterator = bounded_repeater('Hello', 3)

In [185]:
iterator

<generator object bounded_repeater at 0x000002620715D970>

In [186]:
for x in iterator:
    print(x)

Hello
Hello
Hello


In [189]:
iterator = ('Hello' for i in range(3))

In [191]:
for x in iterator:
    print(x)

In [193]:
listcomp = ['Hello' for i in range(3)]
listcomp

['Hello', 'Hello', 'Hello']

In [194]:
genexp = ('Hello' for i in range(3))
genexp

<generator object <genexpr> at 0x0000026208CCD820>

In [195]:
for i in genexp:
    print(i)

Hello
Hello
Hello


In [196]:
# can only use once:
for i in genexp:
    print(i)

In [198]:
genexp = ('Hello' for i in range(3))

In [199]:
next(genexp)

'Hello'

In [200]:
next(genexp)

'Hello'

In [201]:
next(genexp)

'Hello'

In [202]:
# next(genexp)

StopIteration: 

In [203]:
genexp = ('Hello' for i in range(3))

In [204]:
list(genexp)

['Hello', 'Hello', 'Hello']

In [205]:
even_squares = (x * x for x in range(10)
                    if x % 2 == 0)

In [206]:
for x in even_squares:
    print(x)

0
4
16
36
64


In [207]:
for x in ('Bom dia' for i in range(3)):
    print(x)

Bom dia
Bom dia
Bom dia


In [208]:
sum((x * 2 for x in range(10)))

90

In [209]:
sum(x * 2 for x in range(10))

90